In [ ]:
##Import thư viện, mappingpath
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from matplotlib import pyplot as plt
import random
import cv2
from glob import glob
import random

import tensorflow as tf
import keras
from keras.models import *
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adamax
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.applications.efficientnet import EfficientNetB7
from IPython.display import display
from PIL import Image

In [ ]:
#Mapping

mapping_path = '/content/drive/MyDrive/LOC_synset_mapping.txt'
src_path_train = "/content/drive/MyDrive/train_100000"
batch_size = 128

In [ ]:
#PathFile

class_mapping_dict = {}
i = 0
for line in open(mapping_path):
    if 0 <= i < 1000:
        label = line[9:].strip()
        class_mapping_dict[i] = label
    i += 1

# class_mapping_dict = {}
# i = 0
# for line in open(mapping_path):
#     if 1 <= i <= 10:
#         class_mapping_dict[i-1] = line.split(',')[2].strip()
#     i += 1

In [ ]:
image_gen = ImageDataGenerator(
    horizontal_flip=True,
    preprocessing_function = preprocess_input,
    validation_split=0.20)

train_generator = image_gen.flow_from_directory(
  src_path_train,
  target_size=(300,300),
  batch_size=batch_size,
  subset="training",

)

test_generator = image_gen.flow_from_directory(
  src_path_train,
  target_size=(300,300),
  batch_size=batch_size,
  subset="validation",


)

Found 80000 images belonging to 1000 classes.
Found 20000 images belonging to 1000 classes.


In [ ]:
#Tao model
model_efficientnet = EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(300, 300, 3),
    pooling="max"
)

# model_efficientnet.trainable = True
for layer in model_efficientnet.layers:
    layer.trainable = False

x = model_efficientnet.output
x = BatchNormalization()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(len(class_mapping_dict), activation="softmax")(x)

model = Model(inputs=model_efficientnet.input, outputs=outputs)
lr = 0.001

model.compile(optimizer=Adamax(learning_rate=lr), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
## Load lại model

## Đường dẫn đến file model
model_path = "/content/drive/MyDrive/image/momkey_model.h5"

# # Tải model
model = keras.models.load_model(model_path)

In [ ]:
# Train model

from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("/content/drive/MyDrive/DoAnChuyenNganh/modelImagenet.h5", monitor="val_accuracy", save_best_only=True, mode="auto")

n_epochs = 5
hist = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=test_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator),
    callbacks=[ckpoint]
)

Epoch 1/5
180/625 [=======>......................] - ETA: 5:02:19 - loss: 6.1847 - accuracy: 0.0666

KeyboardInterrupt: ignored

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

plot_hist(hist)

In [ ]:
##Test Model
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras


image_url = "/content/drive/MyDrive/image/validation/validation/n0/n000.jpg"
img = Image.open(image_url).resize((300, 300))
img = np.array(img)

pred = model.predict(img[None, :, :])[0]
index = np.argmax(pred)

predicted_class = class_mapping_dict[index]

plt.imshow(Image.open(image_url))
plt.title(predicted_class)
plt.show()
print("Predicted class:", index)
print("Predicted class:", predicted_class)